In [17]:
import pandas as pd
from g4f.client import Client
import re

In [18]:
def load_data(file_path: str) -> pd.DataFrame:
    """
    加載 CSV 文件並處理錯誤行。
    """
    try:
        return pd.read_csv(file_path, on_bad_lines="skip", encoding="utf-8")
    except pd.errors.ParserError as e:
        raise Exception(f"CSV 文件解析錯誤：{e}")
    except Exception as e:
        raise Exception(f"讀取數據時發生錯誤：{e}")



In [19]:
def generate_summary(data: pd.DataFrame):
    """
    生成數據描述性統計。
    Args:
        data (pd.DataFrame): 輸入數據框。
    Returns:
        pd.DataFrame: 描述性統計。
    """
    try:
        return data.describe()
    except Exception as e:
        raise Exception(f"生成描述性統計時發生錯誤：{e}")

In [20]:
def construct_prompt(user_input: str, summary_stats: pd.DataFrame) -> tuple:
    """
    構建系統與用戶提示。
    Args:
        user_input (str): 用戶查詢。
        summary_stats (pd.DataFrame): 描述性統計。
    Returns:
        tuple: (系統提示, 用戶提示)
    """
    system_prompt = (
        "你是一位專業的數據科學家，擅長分析數據並從中提取見解。"
        "以下是一些統計數據，請根據這些數據提供深入分析和建議：\n"
        "1. 描述數據的整體特性，例如平均值、標準差、數據分佈等。\n"
        "2. 尋找可能的異常值或數據問題，並評估其對分析結果的影響。\n"
        "3. 如果有類別型數據，請分析其分佈特性和可能的相關性。\n"
        "4. 根據數據，提供有助於業務決策的具體建議，例如潛在的改進方向或應關注的特定方向。\n"
        "5. 將您的分析結論簡潔清楚地表達，並建議下一步的數據收集或分析方向。"
    )
    user_prompt = (
        f"以下是數據的描述性統計結果：\n{summary_stats.to_string()}\n"
        f"請根據這些數據回答以下問題：{user_input}"
    )
    return system_prompt, user_prompt


In [22]:
def generate_report(file_path: str, user_input: str) -> str:
    """
    基於自然語言查詢生成分析報告。
    Args:
        file_path (str): 數據文件路徑。
        user_input (str): 用戶的自然語言查詢。
    Returns:
        str: 分析報告或錯誤信息。
    """
    try:
        # 1. 加載數據
        data = load_data(file_path)

        # 2. 生成描述性統計
        summary_stats = generate_summary(data)

        # 3. 構建 GPT 模型請求的提示
        system_prompt, user_prompt = construct_prompt(user_input, summary_stats)

        # 4. 初始化 GPT 客戶端並生成報告
        client = Client()
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt},
            ],
            temperature=0.1,
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"生成報告時發生錯誤：{e}"



In [23]:
if __name__ == "__main__":
    # 示例使用
    file_path = file_path = r".\csv.ingredient\tsmc_operating_profit_margin_2010_2023.csv"

    user_input = "請分析數據並提供詳細的見解和建議，並提供報告。"  # 自然語言查詢

    try:
        report = generate_report(file_path, user_input)
        print("生成的報告：")
        print(report)
    except Exception as e:
        print(f"程序運行失敗：{e}")

生成的報告：
### 數據分析報告

#### 1. 整體特性描述
根據提供的描述性統計數據，我們可以對「Operating Profit Margin (%)」進行以下分析：

- **樣本數量**：共有56個觀測值。
- **平均值**：操作利潤率的平均值為38.38%，顯示出整體的盈利能力相對穩定。
- **標準差**：標準差為2.33%，這表明數據的變異性相對較小，利潤率的波動不大。
- **數據範圍**：操作利潤率的最小值為33.60%，最大值為42.81%，顯示出一定的增長潛力。
- **四分位數**：
  - 第一四分位數（25%）為36.39%
  - 中位數（50%）為38.63%
  - 第三四分位數（75%）為40.09%
  
  這些數據表明，隨著時間的推移，操作利潤率有逐漸上升的趨勢。

#### 2. 異常值和數據問題
在這組數據中，沒有明顯的異常值（outliers）顯示出來。所有的數據點都在合理的範圍內，且標準差相對較小，這意味著數據的穩定性較高。然而，進一步的可視化（如箱型圖）可以幫助確認是否存在潛在的異常值。

#### 3. 類別型數據分析
目前提供的數據中僅包含年份和操作利潤率兩個變數，並未包含類別型數據。如果有其他類別型數據（如行業類別、地區等），可以進一步分析其分佈特性及與操作利潤率的相關性。

#### 4. 業務決策建議
根據數據分析結果，以下是幾個具體的建議：

- **持續監控利潤率**：由於操作利潤率在過去幾年中顯示出穩定的增長趨勢，建議企業持續監控這一指標，並分析影響利潤率的內外部因素。
- **尋找增長機會**：考慮進一步分析影響利潤率的因素，例如成本結構、產品組合或市場需求，以尋找潛在的增長機會。
- **市場定位**：如果可能，進一步細分市場，分析不同市場或產品線的利潤率，這將有助於制定更具針對性的業務策略。

#### 5. 下一步建議
- **數據收集**：建議收集更多的類別型數據（如行業、地區、產品類別等），以便進行更深入的分析。
- **可視化分析**：進行數據可視化（如時間序列圖、箱型圖等），以便更直觀地理解數據的分佈和趨勢。
- **進一步的統計分析**：考慮進行回歸分析或其他統計模型，以評估影響操作利潤率的主要因素。

總結來說，這組數據顯示出穩定的操作利潤率，企業應持續關注並尋找增長機會